In [2]:
import pandas as pd
import numpy as np

In [3]:
merge_train = pd.read_csv("./group_data/merge_train.csv", index_col=0)

C:\Users\SBAUser\AppData\Local\Temp\ipykernel_6140\1415123792.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  merge_train = pd.read_csv("./group_data/merge_train.csv", index_col=0)


년도 & 달 & 일 컬럼 생성

In [138]:
merge_train['year'] = [(x//12)+2013 for x in merge_train['date_block_num']]
                        # x를 12로 나눴을 때 몫을 뽑아 2013을 더해 year 생성
merge_train['month'] = [(x % 12) for x in merge_train['date_block_num']]
                        # x를 12로 나눴을 때 나머지를 뽑아 내서 month 생성 (0~11로 표현)
# 월별 (최대 날짜치)
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
merge_train['day'] = merge_train['month'].map(days)
# x를 12로 나눴을 때 나머지를 뽑아 내서 month 생성 (0~11로 표현)

merge_train['month'] = merge_train['month'].apply(lambda x: x+1)
                    
merge_train.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,main_type_id,sub_type_id,city_id,year,month,day
0,0,5,5037,0,19,5,10,3,2013,1,31
1,0,5,5320,0,55,13,2,3,2013,1,31
2,0,5,5233,0,19,5,10,3,2013,1,31
3,0,5,5232,0,23,5,16,3,2013,1,31
4,0,5,5268,0,20,5,11,3,2013,1,31


In [139]:
# n달 전 만큼 feature을 얻을 수 있음.
def get_lag_feature(df, lags:list, columns):
    tmp = df[['date_block_num', 'shop_id', 'item_id', columns]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', columns+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

한달 전 데이터 프레임 추가하기

In [140]:
# 기준없이 월별 상점 아이템으로 판매량
merge_train = get_lag_feature(df=merge_train, lags=[1,2,3,6,12], columns='item_cnt_month')
merge_train.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,main_type_id,sub_type_id,city_id,year,month,day,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0,5,5037,0,19,5,10,3,2013,1,31,NaN,NaN,NaN,NaN,NaN
1,0,5,5320,0,55,13,2,3,2013,1,31,NaN,NaN,NaN,NaN,NaN
2,0,5,5233,0,19,5,10,3,2013,1,31,NaN,NaN,NaN,NaN,NaN
3,0,5,5232,0,23,5,16,3,2013,1,31,NaN,NaN,NaN,NaN,NaN
4,0,5,5268,0,20,5,11,3,2013,1,31,NaN,NaN,NaN,NaN,NaN


특정 feature들의 평균 판매량 구하기

In [141]:
def get_group_feature(df, group:list, column):
    month_mean = df.groupby(group).agg({'item_cnt_month':'mean'})
    month_mean.columns = [column]
    month_mean = month_mean.reset_index()
    return month_mean

In [142]:
# date_block_num을 기준으로 뽑은 cnt_month >> 전체 월별 평균 판매량 확인
date_cnt_month_mean = get_group_feature(merge_train, ['date_block_num'], 'date_cnt_month_mean')


# date_block_num과 item_category를 기준으로 뽑은 cnt_month >> 전체 월 & 가게별 평균 판매량 확인 
date_shop_month_mean = get_group_feature(merge_train, ['date_block_num','shop_id'], 'date_shop_month_mean')

# date_block_num과 item_category를 기준으로 뽑은 cnt_month >> 전체 월 & 아이템별 평균 판매량 확인 
date_item_month_mean = get_group_feature(merge_train, ['date_block_num','item_id'], 'date_item_month_mean')

# date_block_num과 item_category를 기준으로 뽑은 cnt_month >> 전체 월 & 카테고리별 평균 판매량 확인 
date_cat_month_mean = get_group_feature(merge_train, ['date_block_num','item_category_id'], 'date_cat_month_mean')



# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 상점 별 아이템 카테고리 평균 판매량 확인
date_shop_cat_month_mean = get_group_feature(merge_train, ['date_block_num', 'shop_id', 'item_category_id'], 'date_shop_cat_month_mean')

# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 상점 별 아이템 자체 그룹화(main) 평균 판매량 확인
date_shop_type_month_mean = get_group_feature(merge_train, ['date_block_num', 'shop_id', 'main_type_id'], 'date_shop_type_month_mean')

# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 상점 별 아이템 자체 그룹화(sub) 평균 판매량 확인
date_shop_subtype_month_mean = get_group_feature(merge_train, ['date_block_num', 'shop_id', 'sub_type_id'], 'date_shop_subtype_month_mean')



# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 도시 별 평균 판매량 확인
date_city_month_mean = get_group_feature(merge_train, ['date_block_num', 'city_id'], 'date_city_month_mean')

# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 도시 별 아이템 평균 판매량 확인
date_city_item_month_mean = get_group_feature(merge_train, ['date_block_num', 'city_id', 'item_id'], 'date_city_item_month_mean')




# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 아이템 자체 그룹화(main) 평균 판매량 확인
date_type_month_mean = get_group_feature(merge_train, ['date_block_num', 'main_type_id'], 'date_type_month_mean')

# date_block_num과 item_category와 shop_id 기준으로 뽑은 cnt_month >> 전체 월 & 아이템 자체 그룹화(main) 평균 판매량 확인
date_subtype_month_mean = get_group_feature(merge_train, ['date_block_num', 'sub_type_id'], 'date_subtype_month_mean')


In [143]:
merge_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 16 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date_block_num         int64  
 1   shop_id                int64  
 2   item_id                int64  
 3   item_cnt_month         int64  
 4   item_category_id       int64  
 5   main_type_id           int64  
 6   sub_type_id            int64  
 7   city_id                int64  
 8   year                   int64  
 9   month                  int64  
 10  day                    int64  
 11  item_cnt_month_lag_1   float64
 12  item_cnt_month_lag_2   float64
 13  item_cnt_month_lag_3   float64
 14  item_cnt_month_lag_6   float64
 15  item_cnt_month_lag_12  float64
dtypes: float64(5), int64(11)
memory usage: 972.4 MB


In [144]:
def reduce_column_type(df):
    for col in df.columns:
        col_type = df[col].dtype
        col_max = df[col].max()
        
        if str(col_type)[:3] == 'int':
            if col_max <= 127:
                df[col] = df[col].astype("int8")
            elif col_max <= 32767:
                df[col] = df[col].astype('int16')
            else:
                df[col] = df[col].astype('int32')
        
        else:
            if col_max <= 65504.0:
                df[col] = df[col].astype("float16")
            elif col_max <= 3.4028235e+38:
                df[col] = df[col].astype("float32")
            else:
                df[col] = df[col].astype("float64")
    return df

In [145]:
merge_train = reduce_column_type(merge_train)

In [146]:
merge_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 16 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date_block_num         int8   
 1   shop_id                int8   
 2   item_id                int16  
 3   item_cnt_month         int8   
 4   item_category_id       int8   
 5   main_type_id           int8   
 6   sub_type_id            int8   
 7   city_id                int8   
 8   year                   int16  
 9   month                  int8   
 10  day                    int8   
 11  item_cnt_month_lag_1   float16
 12  item_cnt_month_lag_2   float16
 13  item_cnt_month_lag_3   float16
 14  item_cnt_month_lag_6   float16
 15  item_cnt_month_lag_12  float16
dtypes: float16(5), int16(2), int8(9)
memory usage: 221.6 MB


특정 feature들의 평균 판매량 병합

In [147]:
def merge_feature(df, merge_df, key:list, lag:list, column):
    df = pd.merge(df, merge_df, how='left', on = key)
    df[column] = df[column].astype(np.float16)
    df = get_lag_feature(df, lag, column)
    df = df.drop(column, axis=1)
    return df

In [148]:
# 전체 월별 평균 판매량 병합
merge_train = merge_feature(df=merge_train, merge_df=date_cnt_month_mean, key=['date_block_num'], lag=[1], column='date_cnt_month_mean')


#전체 월 & 가게별 평균 판매량 병합
merge_train = merge_feature(df=merge_train, merge_df=date_shop_month_mean, key=['date_block_num','shop_id'], lag=[1,2,3,6,12], column='date_shop_month_mean')

#전체 월 & 아이템별 평균 판매량 병합
merge_train = merge_feature(df=merge_train, merge_df=date_item_month_mean, key=['date_block_num','item_id'], lag=[1,2,3,6,12], column='date_item_month_mean')

#전체 월 & 카테고리별 평균 판매량 확인 
merge_train = merge_feature(df=merge_train, merge_df=date_cat_month_mean, key=['date_block_num','item_category_id'], lag=[1], column='date_cat_month_mean')



#전체 월 & 상점 별 아이템 카테고리 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_shop_cat_month_mean, key=['date_block_num', 'shop_id', 'item_category_id'], lag=[1], column='date_shop_cat_month_mean')

#전체 월 & 상점 별 아이템 자체 그룹화(main) 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_shop_type_month_mean, key=['date_block_num', 'shop_id', 'main_type_id'], lag=[1], column='date_shop_type_month_mean')

#전체 월 & 상점 별 아이템 자체 그룹화(sub) 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_shop_subtype_month_mean, key=['date_block_num', 'shop_id', 'sub_type_id'], lag=[1], column='date_shop_subtype_month_mean')



#전체 월 & 도시 별 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_city_month_mean, key=['date_block_num', 'city_id'], lag=[1], column='date_city_month_mean')

#전체 월 & 도시 별 아이템 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_city_item_month_mean, key=['date_block_num', 'city_id', 'item_id'], lag=[1], column='date_city_item_month_mean')



#전체 월 & 아이템 자체 그룹화(main) 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_type_month_mean, key=['date_block_num', 'main_type_id'], lag=[1], column='date_type_month_mean')

#전체 월 & 아이템 자체 그룹화(main) 평균 판매량 확인
merge_train = merge_feature(df=merge_train, merge_df=date_subtype_month_mean, key=['date_block_num', 'sub_type_id'], lag=[1], column='date_subtype_month_mean')

In [149]:
future_sales = merge_train.fillna(0)

In [154]:
future_sales.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,main_type_id,sub_type_id,city_id,year,month,...,date_item_month_mean_lag_6,date_item_month_mean_lag_12,date_cat_month_mean_lag_1,date_shop_cat_month_mean_lag_1,date_shop_type_month_mean_lag_1,date_shop_subtype_month_mean_lag_1,date_city_month_mean_lag_1,date_city_item_month_mean_lag_1,date_type_month_mean_lag_1,date_subtype_month_mean_lag_1
0,0,5,5037,0,19,5,10,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,5,5320,0,55,13,2,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,5,5233,0,19,5,10,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,5,5232,0,23,5,16,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,5,5268,0,20,5,11,3,2013,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
future_sales = reduce_column_type(future_sales)

In [151]:
future_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 35 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   date_block_num                      int8   
 1   shop_id                             int8   
 2   item_id                             int16  
 3   item_cnt_month                      int8   
 4   item_category_id                    int8   
 5   main_type_id                        int8   
 6   sub_type_id                         int8   
 7   city_id                             int8   
 8   year                                int16  
 9   month                               int8   
 10  day                                 int8   
 11  item_cnt_month_lag_1                float16
 12  item_cnt_month_lag_2                float16
 13  item_cnt_month_lag_3                float16
 14  item_cnt_month_lag_6                float16
 15  item_cnt_month_lag_12               float16
 16  

In [152]:
future_sales.to_csv('future_sales.csv', index=False)

In [153]:
from tqdm import tqdm

for i in tqdm(range(100), leave=True):
    print(i)

100%|██████████| 100/100 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
